je vais manipuler des noms d'auteurs que je vais extraire d'une transcription. la transcription a ete cree a partir d'un enregistrement audio donc le risque est d'avoir des erreurs dans l'orthographe de cet auteur.

Plan for Author Name Handling System
- Create a class to handle author names with fuzzy matching
- Store reference names in a dictionary/database
- Implement fuzzy string matching using Levenshtein distance
- Add methods to suggest corrections for misspelled names
- Include confidence scoring for matches

# thefuzz

In [ ]:
from thefuzz import fuzz
from thefuzz import process
from typing import Dict, List, Tuple


class AuthorMatcher:
    def __init__(self, reference_authors: List[str] = None):
        """Initialize with a list of known author names"""
        self.reference_authors = set(reference_authors) if reference_authors else set()

    def add_reference_author(self, author: str) -> None:
        """Add a new reference author to the set"""
        self.reference_authors.add(author.strip())

    def find_best_match(self, name: str, min_score: int = 80) -> Tuple[str, int]:
        """
        Find the best matching reference author for a given name
        Returns: (best_match, score)
        """
        if not name or not self.reference_authors:
            return None, 0

        # Find best match using token set ratio for better partial matching
        best_match, score = process.extractOne(
            name, self.reference_authors, scorer=fuzz.token_set_ratio
        )

        if score >= min_score:
            return best_match, score
        return None, score

    def suggest_corrections(self, name: str, limit: int = 3) -> List[Tuple[str, int]]:
        """Suggest possible corrections for a name"""
        if not name or not self.reference_authors:
            return []

        # Get top N matches with scores
        matches = process.extract(
            name, self.reference_authors, limit=limit, scorer=fuzz.token_set_ratio
        )

        return matches


# Example usage
if __name__ == "__main__":
    # Initialize with some reference authors
    reference_authors = [
        "Victor Hugo",
        "Albert Camus",
        "Simone de Beauvoir",
        "Marcel Proust",
        "Émile Zola",
    ]

    matcher = AuthorMatcher(reference_authors)

    # Test with a misspelled name
    test_name = "Viktor Hugo"
    best_match, score = matcher.find_best_match(test_name)
    print(f"Input: {test_name}")
    print(f"Best match: {best_match} (score: {score})")

    # Get suggestions
    suggestions = matcher.suggest_corrections(test_name)
    print("\nSuggestions:")
    for name, score in suggestions:
        print(f"- {name} (score: {score})")

Input: Viktor Hugo
Best match: Victor Hugo (score: 91)

Suggestions:
- Victor Hugo (score: 91)
- Simone de Beauvoir (score: 34)
- Émile Zola (score: 20)


pas mal mais l'inconvenient c'est qu'il faut les auteurs de reference

# avec un llm pour obtenir la liste d'auteurs de reference

In [ ]:
from llm import get_azure_llm

llm = get_azure_llm()

In [ ]:
response = llm.complete("Explain how AI works")
print(response.text[:100])

Artificial Intelligence (AI) is a broad field of computer science focused on creating systems capabl


In [ ]:
from llama_index.core.llms import ChatMessage, ChatResponse
import json

llm = get_azure_llm("gpt-4o-mini")

response_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "AuthorList",
        "schema": {
            "type": "object",
            "properties": {
                "Authors_by_Guillaume": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "description": "A list of authors' names",
                    },
                }
            },
            "required": ["Authors_by_Guillaume"],
            "additionalProperties": False,
        },
    },
}


def get_authors_json(query):
    response = llm.chat(
        messages=[
            ChatMessage(
                role="system",
                content="You are a helpful assistant that returns a JSON list of strings.",
            ),
            # ChatMessage(role="user", content=f"{query}. Please provide the response in JSON format as a list of strings, following this schema: ['author1', 'author2', ...]")
            ChatMessage(role="user", content=f"{query}. "),
        ],
        response_format=response_schema,
    )
    return response


# Exemple d'utilisation
query = "List 5 famous science fiction authors"
json_response = get_authors_json(query)

# print("Raw response:", json_response)  # Debug

try:
    json_dict = json.loads(json_response.message.content)
    # ...
except json.JSONDecodeError as e:
    print("Error parsing JSON:", e)
    print("Raw response:", json_response)
json_dict

{'Authors_by_Guillaume': ['Isaac Asimov',
  'Arthur C. Clarke',
  'Philip K. Dick',
  'Ursula K. Le Guin',
  'Ray Bradbury']}

In [ ]:
from thefuzz import fuzz
from thefuzz import process
from llm import get_azure_llm
from typing import List, Tuple
import json
from pathlib import Path


class AuthorMatcher:
    def __init__(self, cache_file: str = "french_authors_cache.json"):
        """Initialize with gemini client and cache"""
        self.client = get_azure_llm()
        self.cache_file = Path(cache_file)
        self.reference_authors = self._load_or_fetch_authors()

    def _load_or_fetch_authors(self) -> List[str]:
        """Load authors from cache or fetch from OpenAI"""
        if self.cache_file.exists():
            with open(self.cache_file, "r") as f:
                return json.load(f)

        # Fetch from OpenAI if cache doesn't exist
        prompt = """
        You are a literary expert assistant.

        Give me a list of 100 important French authors, including contemporary ones, 
        that might be discussed in "Le Masque et la Plume". 
        Return only a JSON array of names, no other text.
        """

        response = self.client.complete(prompt)

        try:
            # Extract JSON from response text
            # Remove any markdown formatting or extra text
            response_text = response.text.strip()
            if response_text.startswith("```json"):
                response_text = response_text.split("```json")[1]
            if response_text.endswith("```"):
                response_text = response_text.split("```")[0]

            authors = json.loads(response_text.strip())

            # Validate response format
            if not isinstance(authors, list):
                raise ValueError("Response is not a list")
            if not all(isinstance(x, str) for x in authors):
                raise ValueError("Not all elements are strings")

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
            print(f"Raw response: {response.text}")
            return []

        # complete avec la logique de cache
        with open(self.cache_file, "w") as f:
            json.dump(authors, f)

        return authors

    def find_best_match(self, name: str, min_score: int = 80) -> Tuple[str, int]:
        """Find best matching reference author"""
        if not name:
            return None, 0

        best_match, score = process.extractOne(
            name, self.reference_authors, scorer=fuzz.token_set_ratio
        )

        if score >= min_score:
            return best_match, score
        return None, score

    def suggest_corrections(self, name: str, limit: int = 3) -> List[Tuple[str, int]]:
        """Suggest possible corrections for a name"""
        return process.extract(
            name, self.reference_authors, limit=limit, scorer=fuzz.token_set_ratio
        )


def test_auteur(autor):
    matcher = AuthorMatcher()
    best_match, score = matcher.find_best_match(autor)
    print(f"Input: {autor}")
    print(f"Best match: {best_match} (score: {score})")
    suggestions = matcher.suggest_corrections(autor)
    print("\nSuggestions:")
    for name, score in suggestions:
        print(f"- {name} (score: {score})")


test_auteur("Viktor Hugo")

Input: Viktor Hugo
Best match: Victor Hugo (score: 91)

Suggestions:
- Victor Hugo (score: 91)
- Victor Segalen (score: 56)
- Siri Hustvedt (score: 42)


le probleme c'est que si j'utilise un auteur moins connu qui n'est pas dans les 100 ca ne va pas marcher

In [ ]:
test_auteur("Frederic Beigbeder")

Input: Frederic Beigbeder
Best match: None (score: 45)

Suggestions:
- Georges Perec (score: 45)
- Simone de Beauvoir (score: 44)
- Pierre Corneille (score: 41)


et avec des accents ?

In [ ]:
test_auteur("Gael Faye")

Input: Gael Faye
Best match: Gaël Faye (score: 94)

Suggestions:
- Gaël Faye (score: 94)
- Gustave Flaubert (score: 56)
- Françoise Sagan (score: 52)


ca ca marche

# avec un llm pour retourner les auteurs dont le nom ressemble

In [ ]:
from thefuzz import fuzz
from thefuzz import process
from llm import get_azure_llm
from typing import List, Tuple
import json
from pathlib import Path


class AuthorMatcher:
    def __init__(self):
        """Initialize with gemini client"""
        self.client = get_azure_llm()

    def _fetch_author(self, autor) -> List[str]:
        """fetch from gemini"""

        # Fetch from gemini
        prompt = (
            """
        Tu es un agent expert en littérature.
        Donne moi quelques auteurs dont le nom s'approche de celui-ci : """
            + autor
            + """

        S'il s'agit deja d'un auteur connu, retourne moi juste son nom. S'il y a une erreur dans le nom que je t'ai donne, corrige moi en me donnant le nom de l'auteur que tu penses que j'ai voulu dire.

        Je veux que tu me donnes le prenom puis le nom dans cet ordre. Par exemple "Marcel Pagnol" ou "Victor Hugo".
        Ces auteurs sont susceptibles d'etre discutes dans "Le Masque et la Plume".

        Si tu me retournes plusieurs auteurs, fais le sous forme de liste par exemple si tu as identifie "auteur 1" et "auteur 2" alors retourne ["auteur 1", "auteur 2"]

        Retourne uniquement un tableau JSON de noms, pas de texte supplémentaire.
        """
        )

        response = self.client.complete(prompt)

        try:
            # Extract JSON from response text
            # Remove any markdown formatting or extra text
            response_text = response.text.strip()
            if response_text.startswith("```json"):
                response_text = response_text.split("```json")[1]
            if response_text.endswith("```"):
                response_text = response_text.split("```")[0]

            authors = json.loads(response_text.strip())

            # Validate response format
            if not isinstance(authors, list):
                raise ValueError("Response is not a list")
            if not all(isinstance(x, str) for x in authors):
                print(f"Raw response: {response.text}")
                raise ValueError("Not all elements are strings")

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
            print(f"Raw response: {response.text}")
            return []

        return authors

    def find_best_match(self, name: str, min_score: int = 80) -> Tuple[str, int]:
        """Find best matching reference author"""
        if not name:
            return None, 0

        fetch_author = self._fetch_author(name)

        if not fetch_author:
            return None, 0

        best_match, score = process.extractOne(
            name, fetch_author, scorer=fuzz.token_set_ratio
        )

        if score >= min_score:
            return best_match, score
        return None, score

    def suggest_corrections(self, name: str, limit: int = 3) -> List[Tuple[str, int]]:
        """Suggest possible corrections for a name"""
        fetch_author = self._fetch_author(name)

        if not fetch_author:
            return []
        return process.extract(
            name, fetch_author, limit=limit, scorer=fuzz.token_set_ratio
        )


def test_auteur(autor):
    matcher = AuthorMatcher()
    best_match, score = matcher.find_best_match(autor)
    print(f"Input: {autor}")
    print(f"Best match: {best_match} (score: {score})")
    suggestions = matcher.suggest_corrections(autor)
    print("\nSuggestions:")
    for name, score in suggestions:
        print(f"- {name} (score: {score})")


test_auteur("Viktor Hugo")

Input: Viktor Hugo
Best match: Victor Hugo (score: 91)

Suggestions:
- Victor Hugo (score: 91)


In [ ]:
test_auteur("Frederic Beigbeder")

Input: Frederic Beigbeder
Best match: Frederic Beigbeder (score: 100)

Suggestions:
- Frédéric Beigbeder (score: 94)


est-ce que ca marche avec des auteurs recents ? premiers romans ?

In [ ]:
test_auteur("Jeanne Rivière")

Input: Jeanne Rivière
Best match: None (score: 61)

Suggestions:
- Jean Rolin (score: 61)
- Jean Rouaud (score: 50)
- Jean Echenoz (score: 32)


In [ ]:
matcher = AuthorMatcher()
matcher._fetch_author("Jeanne Rivière")
# matcher._fetch_author("Frederic Beigbeder")

['Jean Rolin', 'Jean Rouaud', 'Jean Echenoz']

ca ne marche pas du tout avec les auteurs recents

# en fournissant la description de l'episode

on va tester avec l'episode du 26 janvier 2025 

et "Jeanne Rivière" qui vient d'ecrire son 1er roman

In [ ]:
from mongo_episode import Episode
import datetime

episode_26janv2025 = Episode.from_date(datetime.date(2025, 1, 26))

In [ ]:
episode_26janv2025.description

"durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France\xa0; une histoire d'amour et une réflexion sur la judéité\xa0; un roman filial et d'espionnage dans la Guerre Froide\xa0; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault"

In [ ]:
from thefuzz import fuzz
from thefuzz import process
from llm import get_azure_llm
from typing import List, Tuple
import json
from pathlib import Path


class AuthorEpisodeMatcher:
    def __init__(self, episode: Episode):
        """un agent qui va verifier les auteurs d'un episode
        episode : episode qui contient notamment un titre et une description
        """
        self.client = get_azure_llm()
        self.episode = episode

    def _potentiels_auteurs(self, auteur: str) -> List[str]:
        """
        on cherche avec llm une liste de potentiels auteurs qui pourraient correspondre à l'auteur donné
        auteur : ce qu'on cherche

        cette liste sera utilise par find_best_match pour trouver le meilleur match
        """

        description = (
            " titre : " + self.episode.titre
        )  # + " \n description : " + self.episode.description

        # prompt pour llm
        prompt = (
            """
        Tu es un agent expert en littérature.

        J'ai entendu parler d'un auteur evoque dans un episode du "Masque et la Plume" dont le nom ressemble à celui-ci : """
            + auteur
            + """

        Je dis ressemble parce que j'ai entendu le nom à la radio et je ne suis pas sûr de l'orthographe exacte de son nom.

        J'ai aussi le titre et lq description de cet episode du "Masque et la Plume" qui peuvent t'aider à trouver cet auteur. 
        Les voici : """
            + description
            + """

        Il est possible que le nom apparaisse dans le titre ou la description, dans ce cas utilise le car ces sources sont plus fiables que le nom que je t'ai donne.
        
        Cependant il est possible que le nom n'y apparaisse pas, dans ce cas recherche dans ce que tu connais en tant qu'expert en littérature.
        
        S'il s'agit deja d'un auteur que tu connaissais, retourne moi juste son nom. S'il y a une erreur dans le nom que je t'ai donne, corrige moi en me donnant le nom de l'auteur que tu penses que j'ai voulu dire.

        Je veux que tu me donnes le prenom puis le nom dans cet ordre. Par exemple "Marcel Pagnol" ou "Victor Hugo".

        Si tu me retournes plusieurs auteurs car tu as des doutes, fais le sous forme de liste par exemple si tu as identifie "auteur 1" et "auteur 2" alors retourne ["auteur 1", "auteur 2"]

        Retourne uniquement un tableau JSON de noms, pas de texte supplémentaire.
        """
        )

        response = self.client.complete(prompt)

        try:
            # Extract JSON from response text
            # Remove any markdown formatting or extra text
            response_text = response.text.strip()
            if response_text.startswith("```json"):
                response_text = response_text.split("```json")[1]
            if response_text.endswith("```"):
                response_text = response_text.split("```")[0]

            authors = json.loads(response_text.strip())

            # Validate response format
            if not isinstance(authors, list):
                raise ValueError("Response is not a list")
            if not all(isinstance(x, str) for x in authors):
                print(f"Raw response: {response.text}")
                raise ValueError("Not all elements are strings")

        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
            print(f"Raw response: {response.text}")
            return []

        return authors

    def find_best_match(self, auteur: str, min_score: int = 80) -> Tuple[str, int]:
        """Find best matching reference auteur
        en utilisant thefuzz process.extractOne

        retourne None, 0 si aucun auteur n'est renseigné
        retourne None, -1 si aucun auteur de reference n'est trouvé
        retourne None, score si le score est inferieur à min_score

        et sinon retourne le meilleur match et son score

        Returns: (best_match, score)
        """
        if not auteur:
            return None, 0

        fetch_author = self._potentiels_auteurs(auteur)

        if not fetch_author:
            return None, -1

        best_match, score = process.extractOne(
            auteur, fetch_author, scorer=fuzz.token_set_ratio
        )

        if score >= min_score:
            return best_match, score
        return None, score

    def test_auteur(self, autor):
        best_match, score = self.find_best_match(autor)
        print(f"Input: {autor}")
        print(f"fetch autors : {self._potentiels_auteurs(autor)}")
        print(f"Best match: {best_match} (score: {score})")
        return best_match, score


matcher_26janv2025 = AuthorEpisodeMatcher(episode_26janv2025)

In [ ]:
matcher_26janv2025.test_auteur("Viktor Hugo")

Input: Viktor Hugo
fetch autors : ['Leïla Slimani', 'Pierre Lemaître', 'Jeanne Rivière', 'Nathalie Azoulai', 'Milena Agus']
Best match: Victor Hugo (score: 91)


('Victor Hugo', 91)

In [ ]:
matcher_26janv2025.test_auteur("Jeanne Rivière")

Input: Jeanne Rivière
fetch autors : ['Jeanne Benameur']
Best match: None (score: 63)


(None, 63)

In [ ]:
matcher_26janv2025.test_auteur("Jeanne Riviere")

Input: Jeanne Riviere
fetch autors : ['Jeanne Rivière']
Best match: Jeanne Rivière (score: 96)


('Jeanne Rivière', 96)

c'est pas parfait c'est le cote un peu schostastique du llm

essayons avec la transcription

In [ ]:
transcription = episode_26janv2025.transcription

prompt_transcription = f"""

Je vais te donner la transcription d'un episode d'une emission de radio qui s'appelle le masque et la plume sur France Inter.
Cet episode dure 1h et porte sur des livres. Il y a des intervenants qui parlent des livres qu'ils ont lus. Ils ne sont parfois pas d'accord.

Voici la transcription:
{transcription}

Je veux que tu identifies l'ensemble des livres dont on parle dans cette emission.
Et que tu me restitues cette liste de livres en separant auteur et titre. Si l'editeur est mentionne tu peux aussi le noter.

Tu me restitueras cette liste sous la forme d'un tableau au format markdown. Avec une colonne pour l'auteur, une colonne pour le titre et une colonne pour l'editeur si il est mentionne.

Et tu me donneras une lsite au format python des auteurs
"""

In [ ]:
llm = get_azure_llm()

llm.complete(prompt_transcription).text

'Voici la liste des livres mentionnés dans l\'émission, présentée sous forme de tableau au format markdown :\n\n| Auteur                | Titre                                    | Éditeur          |\n|-----------------------|------------------------------------------|------------------|\n| Leïla Slimani         | J\'emporterai le feu                      | Gallimard        |\n| Nathalie Azoulay      | Toutes les vies de Théo                  | POL              |\n| Pierre Lemaitre       | Un avenir radieux                        | Calman Lévy      |\n| Jeanne Rivière        | Lorraine Brulle                          | Gallimard        |\n| Milena Agus           | Le vent passe et la nuit aussi           | Liana Lévy       |\n| Christian Laval       | Marx en Amérique                         | Chamballon       |\n| Guillaume Lebrun      | Ravager de splendeur                     | Bourgois         |\n| Sally Rooney          | Normal People                            | Livre de poche   

'Voici le tableau markdown listant les livres mentionnés dans l\'émission, avec auteur, titre et éditeur :\n\n

| Auteur             | Titre                         | Éditeur           |
|----------------------|---------------------------------|--------------------|
| Haruki Murakami     | La Cité aux murs incertains      |                    |
| Constantin Alexandrakis | L\'Hospitalité au démon         |                    |
| Leila Slimani        | J\'emporterai le feu            | Gallimard          |
| Nathalie Azoulay     | Toutes les vies de Théo        | POL                |
| Johann Svar          | (Titre non spécifié)            |                    |
| Pierre Lemaitre     | Un avenir radieux              | Calmann Lévy       |
| Jeanne Rivière       | Lorraine Brulle                | Gallimard (collection Signe) |
| Milena Agus          | Le vent passe et la nuit aussi | Liana Lévy         |
| Christian Laval      | Marx en Amérique               | Chambon             |
| Guillaume Lebrun     | Ravagé de splendeur            | Bourgois           |
| Sally Rooney         | Normal People                  | Livre de Poche     |
| Jean-Patrick Manchette | (Titre non spécifié, biographie)| Gallimard          |
| Bob Dylan            | (Titre non spécifié, biographie)|                    |


Voici la liste des auteurs au format Python

```python
auteurs = ["Haruki Murakami", "Constantin Alexandrakis", "Leila Slimani", "Nathalie Azoulay", "Johann Svar", "Pierre Lemaitre", "Jeanne Rivière", "Milena Agus", "Christian Laval", "Guillaume Lebrun", "Sally Rooney", "Jean-Patrick Manchette", "Bob Dylan"]
```

In [ ]:
auteurs = [
    "Haruki Murakami",
    "Constantin Alexandrakis",
    "Leila Slimani",
    "Nathalie Azoulay",
    "Johann Svar",
    "Pierre Lemaitre",
    "Jeanne Rivière",
    "Milena Agus",
    "Christian Laval",
    "Guillaume Lebrun",
    "Sally Rooney",
    "Jean-Patrick Manchette",
    "Bob Dylan",
]

auteurs_corrects = []
auteurs_inconnus = []

for auteur in auteurs:
    best_match, score = matcher_26janv2025.test_auteur(auteur)
    if score >= 80:
        auteurs_corrects.append(best_match)
    else:
        auteurs_inconnus.append(auteur)

print("Auteurs corrects : ", auteurs_corrects)
print("Auteurs inconnus : ", auteurs_inconnus)

Input: Haruki Murakami
fetch autors : ['Haruki Murakami']
Best match: Haruki Murakami (score: 100)
Input: Constantin Alexandrakis
fetch autors : []
Best match: None (score: -1)
Input: Leila Slimani
fetch autors : ['Leïla Slimani']
Best match: Leïla Slimani (score: 96)
Input: Nathalie Azoulay
fetch autors : ['Nathalie Azoulai']
Best match: Nathalie Azoulai (score: 94)
Input: Johann Svar
fetch autors : ['Johann Zarca']
Best match: None (score: 78)
Input: Pierre Lemaitre
fetch autors : ['Pierre Lemaitre']
Best match: Pierre Lemaitre (score: 100)
Input: Jeanne Rivière
fetch autors : ['Jeanne Benameur']
Best match: None (score: 63)
Input: Milena Agus
fetch autors : ['Leïla Slimani', 'Pierre Lemaître', 'Jeanne Rivière', 'Nathalie Azoulai', 'Milena Agus']
Best match: Milena Agus (score: 100)
Input: Christian Laval
fetch autors : ['Leïla Slimani', 'Pierre Lemaître', 'Jeanne Rivière', 'Nathalie Azoulai', 'Milena Agus']
Best match: None (score: 32)
Input: Guillaume Lebrun
fetch autors : ['Guilla

In [ ]:
from serpapi import GoogleSearch
from llm import load_env
import os

load_env()
serpapi_key = os.getenv("SERP_API_KEY")

params = {
    "api_key": serpapi_key,
    "engine": "google",
    "q": auteurs_inconnus[0],
    "location": "Paris, France",
    "google_domain": "google.com",
    "gl": "fr",
    "hl": "fr",
}

search = GoogleSearch(params)
results = search.get_dict()

In [ ]:
prompt_incertitude_auteur = f"""

Voici le resultat d'une requete google concernant un probable auteur inconnu de mon llm : {auteurs_inconnus[0]}

La requete est au format dict avec du json a l'interieur.

Est-ce que tu peux analyser le contenu de cette requete et me dire si oui ou non {auteurs_inconnus[0]} est un auteur de livres, et accompagner ta reponse d'un pourcentage de certitude

100% de certitude signifie que tu es certain que {auteurs_inconnus[0]} est un auteur de livres

50% tu es ni sure ni pas sure que {auteurs_inconnus[0]} est un auteur de livres

0% tu es certain que {auteurs_inconnus[0]} n'est pas un auteur de livres

Voici le contenu de la requete google : {results}


Tu repondras uniquement avec un dictionnaire qui va contenir 3 entrees :

- "auteur" : le nom de l'auteur, eventuellement corrige si j'ai oublie des accents ou une faute de frappe
- "certitude" : le pourcentage de certitude de 0 à 100, un entier
- "analyse" : une analyse de la requete google

je ne veux pas d'autres choses que ce dictionnaire car je veux utiliser telle quelle ta reponse pour la suite de mon programme ecrit en python

ca veut dire que tu dois commencer ta reponse par un "{" et la terminer par un "}" et que tu ne dois pas ajouter de texte supplementaire

"""

answer = llm.complete(prompt_incertitude_auteur).text

In [ ]:
answer

'{\n  "auteur": "Constantin Alexandrakis",\n  "certitude": 100,\n  "analyse": "La requête Google indique clairement que Constantin Alexandrakis est un auteur. Il est mentionné comme autobiographe dans le knowledge graph et plusieurs résultats de recherche font référence à ses livres, notamment \'L\'hospitalité au démon\' et \'Deux fois né\'. Les liens vers des sites comme Gallimard, Babelio et des articles de presse confirment son statut d\'auteur."\n}'

In [ ]:
import ast

answer_dict = ast.literal_eval(answer)
answer_dict["certitude"], answer_dict["auteur"]

(100, 'Constantin Alexandrakis')

In [ ]:
import os
from googleapiclient.discovery import build
from google.oauth2 import service_account

from llm import load_env
import os

load_env()
api_key = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
cse_id = os.getenv("SEARCH_ENGINE_ID")

if not api_key or not cse_id:
    raise ValueError(
        "Les variables d'environnement GOOGLE_SEARCH_API_KEY et GOOGLE_CSE_ID doivent être définies."
    )


# Fonction de recherche Google
def google_search(query):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=query, cx=cse_id).execute()

        results = []
        for item in res.get("items", []):
            title = item.get("title")
            snippet = item.get("snippet")
            link = item.get("link")
            results.append({"title": title, "snippet": snippet, "link": link})
        return results
    except Exception as e:
        print(f"Erreur lors de la recherche Google: {e}")
        return None


# Exemple d'utilisation
user_query = auteurs_inconnus[0]
response = google_search(user_query)
print(response)

[{'title': 'Constantine G. Alexandrakis - Russell Reynolds Associates | LinkedIn', 'snippet': 'Constantine Alexandrakis is the Chief Executive Officer of Russell Reynolds Associates… · Experience: Russell Reynolds Associates · Location: Greater\xa0...', 'link': 'https://www.linkedin.com/in/constantine-g-alexandrakis'}, {'title': 'Constantine Alexandrakis | CEO | Russell Reynolds Associates', 'snippet': "Constantine Alexandrakis is the Chief Executive Officer of Russell Reynolds Associates and serves on the firm's Board of Directors.", 'link': 'https://www.russellreynolds.com/en/people/consultant-directory/constantine-alexandrakis'}, {'title': 'Constantine Alexandrakis - Faculty Bio | Hofstra University', 'snippet': 'Bio. Constantine Alexandrakis received his doctoral degree in Economics at Clark University. Before joining Hofstra in 2004 he held the position of Assistant\xa0...', 'link': 'https://www.hofstra.edu/faculty-staff/faculty-profile.html?id=26'}, {'title': 'Constantin Alexandr

In [ ]:
results = google_search(auteurs_inconnus[0])

prompt_incertitude_auteur = f"""

Voici le resultat d'une requete google concernant un probable auteur inconnu de mon llm : {auteurs_inconnus[0]}

La requete est au format dict avec du json a l'interieur.

Est-ce que tu peux analyser le contenu de cette requete et me dire si oui ou non {auteurs_inconnus[0]} est un auteur de livres, et accompagner ta reponse d'un pourcentage de certitude

100% de certitude signifie que tu es certain que {auteurs_inconnus[0]} est un auteur de livres

50% tu es ni sure ni pas sure que {auteurs_inconnus[0]} est un auteur de livres

0% tu es certain que {auteurs_inconnus[0]} n'est pas un auteur de livres

Voici le contenu de la requete google : {results}


Tu repondras uniquement avec un dictionnaire qui va contenir 3 entrees :

- "auteur" : le nom de l'auteur, eventuellement corrige si j'ai oublie des accents ou une faute de frappe
- "certitude" : le pourcentage de certitude de 0 à 100, un entier
- "analyse" : une analyse de la requete google

je ne veux pas d'autres choses que ce dictionnaire car je veux utiliser telle quelle ta reponse pour la suite de mon programme ecrit en python

ca veut dire que tu dois commencer ta reponse par un "{" et la terminer par un "}" et que tu ne dois pas ajouter de texte supplementaire
"""

answer = llm.complete(prompt_incertitude_auteur).text

import ast

answer_dict = ast.literal_eval(answer)
answer_dict["certitude"], answer_dict["auteur"]

(90, 'Constantin Alexandrakis')